In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import sys

model_path = '/language-style-transfer/code'

if model_path not in sys.path:
    sys.path.append(model_path)

In [3]:
from vocab import Vocabulary, build_vocab
from accumulator import Accumulator
from options import load_arguments
from file_io import load_sent, write_sent
from utils import *
from nn import *
import beam_search, greedy_decoding

In [4]:
from style_transfer import Model, transfer, create_model

In [5]:
from itertools import groupby
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf

In [6]:
pd.set_option('display.max_colwidth', 999)

In [61]:
from collections import namedtuple
args = {
    'batch_size': 32,
    'beam': 1,
    'dev': '/storage/data3/ods_dota_dev',
    'dim_emb': 100,
    'dim_y': 200,
    'dim_z': 500,
    'dropout_keep_prob': 0.5,
    'embedding': '',
    'filter_sizes': '1,2,3,4,5',
    'gamma_decay': 1,
    'gamma_init': 0.1,
    'gamma_min': 0.1,
    'learning_rate': 0.0005,
    'load_model': True,
    'max_epochs': 20,
    'max_seq_length': 10,
    'max_train_size': -1,
    'model': '/storage/tmp/model',
    'n_filters': 128,
    'n_layers': 1,
    'online_testing': False,
    'output': '/storage/tmp/ods_dota.dev',
    'rho': 1,
    'steps_per_checkpoint': 1000,
    'test': '',
    'train': '/storage/data3/ods_dota',
    'vocab': '/storage/tmp/ods_dota.vocab'
}

args = namedtuple('args', args.keys())(*args.values())

In [12]:
from collections import OrderedDict

def remove_duplicates(tokens):
    return [g[0] for g in groupby(tokens)]

def decode_sentences(sentences):
    return [' '.join(remove_duplicates(tokens)) for tokens in sentences]

def transfer(model, decoder, sess, args, vocab, data0, data1):
    batches, order0, order1 = get_batches(data0, data1,
        vocab.word2id, args.batch_size)

    #data0_rec, data1_rec = [], []
    data0_tsf, data1_tsf = [], []
    losses = Accumulator(len(batches), ['loss', 'rec', 'adv', 'd0', 'd1'])
    for batch in batches:
        rec, tsf = decoder.rewrite(batch)
        half = batch['size'] / 2
        #data0_rec += rec[:half]
        #data1_rec += rec[half:]
        data0_tsf += tsf[:half]
        data1_tsf += tsf[half:]

        loss, loss_rec, loss_adv, loss_d0, loss_d1 = sess.run([model.loss,
            model.loss_rec, model.loss_adv, model.loss_d0, model.loss_d1],
            feed_dict=feed_dictionary(model, batch, args.rho, args.gamma_min))
        losses.add([loss, loss_rec, loss_adv, loss_d0, loss_d1])

    n0, n1 = len(data0), len(data1)
    #data0_rec = reorder(order0, data0_rec)[:n0]
    #data1_rec = reorder(order1, data1_rec)[:n1]
    data0_tsf = reorder(order0, data0_tsf)[:n0]
    data1_tsf = reorder(order1, data1_tsf)[:n1]

    return losses, data0_tsf, data1_tsf

In [149]:
vocab = Vocabulary(args.vocab, args.embedding, args.dim_emb)

In [168]:
tf.reset_default_graph()

config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

sess = tf.InteractiveSession(config=config)

model = create_model(sess, args, vocab)

Loading model from /storage/tmp/model
INFO:tensorflow:Restoring parameters from /storage/tmp/model


In [169]:
if args.beam > 1:
    decoder = beam_search.Decoder(sess, args, vocab, model)
else:
    decoder = greedy_decoding.Decoder(sess, args, vocab, model)

In [160]:
test0 = [sentence.split(' ') for sentence in ['ты нахуя страты палишь буржуй .'] * args.batch_size]
test1 = [sentence.split(' ') for sentence in ['ты нахуя страты палишь буржуй .'] * args.batch_size]

In [170]:
test0 = list(np.random.choice(load_sent(args.dev + '.0'), args.batch_size))
test1 = list(np.random.choice(load_sent(args.dev + '.1'), args.batch_size))

In [171]:
losses, tsf0, tsf1 = transfer(model, decoder, sess, args, vocab, test0, test1)

In [172]:
losses.output()

loss 21.05, rec 16.44, adv 4.61, d0 1.46, d1 1.77


In [173]:
print('ODS -> DotA:')
for s1, s2 in zip(decode_sentences(test0), decode_sentences(tsf0)):
    print("%s\n%s\n" % (s1, s2))

print('DotA -> ODS:')
for s1, s2 in zip(decode_sentences(test1), decode_sentences(tsf1)):
    print("%s\n%s\n" % (s1, s2))

ODS -> DotA:
всё в москве всё в _unk_ _
всё в доте всё _unk_ в _ .

у меня где-то лежит самый топовый рюкзак _unk_
у меня мать самый такой топовый _unk_ .

“ объясню тебе бустинг в очереди на _unk_ ”
пп тебе повезло в лесу на аксе _unk_ сосать .

моя карта по идее
моя карта по идее как бы медом .

seriously though смотри urban dictionary
<unk> смотри блять ) изи мид .

никто из топа не улучшил скор за последние три дня
никто из хуй не нравится 2 дня 2 дня .

но там тогда самый новый элемент не получить
давай там новый очко не успел получить .

для галочки посмотрел можно _unk_ пропускать
мать на рики _unk_ можно армлет ебаный .

это js фреймворки меняются
это закуп ебем или пуле мы тоже .

сгореть ага . прямо как _unk_ пятнадцать лет назад
) как прямо _unk_ лайк лет назад .

взял бы да перевёл на _unk_ основе
да взял бы на _unk_ репорт днище .

дача для быдла
лохи для меня ? .

туплю под вечер
ебать под тавер ) серьёзно . ) .

это винда небось
это там небось ? тоже началось .

сортирую